In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

- request.get()으로 데이터를 가져온 경우

In [2]:
# https://search.naver.com/search.naver?where=view&sm=tab_jum&query=%EC%88%98%EC%9B%90%EB%A7%9B%EC%A7%91
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query=' + quote('수원맛집')
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [3]:
lis = soup.select('.bx._svp_item')
len(lis)

30

- Selenium을 이용해서 데이터 가져오기

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [5]:
# import warnings
# warnings.filterwarnings('ignore')

In [6]:
# from selenium import webdriver
# driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')
driver.get(url)

In [7]:
# View page를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [8]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

110

In [9]:
driver.close()

- 날짜, 제목, 내용 크롤링

In [10]:
li = lis[0]
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2023.04.05.'

In [11]:
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'일품이던 수원맛집베스트10'

In [13]:
contents = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
contents[:200]

'하러 수원맛집베스트10에 속한다는 갈비집을 다녀왔어요. 동네 주민들 뿐만 아니라 멀리서도 많은 사람들이 찾아가는 곳이라는데 소문대로 고퀄리티의 식사를 즐길 수 있던 곳이었어요. 고반가든 서수원점 주소 : 경기...'

- 날짜 처리

In [14]:
from datetime import datetime, timedelta
today = datetime.now()
today

datetime.datetime(2023, 4, 25, 12, 11, 3, 980788)

In [18]:
# 어제
date = today - timedelta(1)
date.strftime('%Y.%m.%d')

'2023.4.24'

In [22]:
# 23시간 전
data = today - timedelta(hours=23)
date.strftime('%Y.%m.%d')

'2023.04.24'

In [23]:
# 2일전
data = today - timedelta(2)
date.strftime('%Y.%m.%d')

'2023.04.24'

In [24]:
# 45분전
data = today - timedelta(minutes=45)
date.strftime('%Y.%m.%d')

'2023.04.24'

In [27]:
def convert_date(s):
    if len(s) == 11:
        return s[:-1]
    today = datetime.now()
    if '어제' in s:
        date = today - timedelta(1)
    elif '일' in s:
        date = today - timedelta(int(s[:-3]))
    elif '시간' in s:
        date = today - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date = today - timedelta(minutes=int(s[-3]))
    return date.strftime('%Y.%m.%d')



In [28]:
lines = []
for li in lis:
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    date = convert_date(date)
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    contents = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    lines.append({'date':date,'title':title,'contents':contents})
df = pd.DataFrame(lines)
df.head()

,date,title,contents
0,2023.04.05,일품이던 수원맛집베스트10,하러 수원맛집베스트10에 속한다는 갈비집을 다녀왔어요. 동네 주민들 뿐만 아니라 멀...
1,2022.05.02,럭셔리한 수원 맛집,후식까지 훌륭했던 수원 맛집이라 조만간 가족들과도 또 와보려고 해요. 2. 행궁빙수...
2,2023.04.23,수원 맛집 영통역 고기집 육풍 영통프라임점 맛있어,아이와 주말에 정말 맛있게 점심 먹었네요 ❤ 수원 맛집 육풍 영통프라임점 ✔ 경기 ...
3,2023.04.24,수원 맛집 신라갈비 한우생갈비의 품격,유명 #수원맛집 후기입니다. 이날 다녀온 신라갈비는 가보정과 본수원갈비와 함께 전국...
4,2022.05.20,호기롭던 수원 맛집,달짝지근한 아이스크림과 바디감 높은 아메리카노로 마무리까지 확실했던 수원 맛집이었어...
